# Logging Examples

The following cells shows examples of how to use the various logging modules and functions. It also will display the outputs of each function.

In [71]:
# import modules

Import-Module -Name ".\cmdlets\Rad-Error-Utils.psm1" -Force
Import-Module -Name ".\cmdlets\Rad-Text-Utils.psm1" -Force

if ($null -eq (Get-Module -ListAvailable -Name powershell-yaml)) {
    Install-Module -Name powershell-yaml -Scope CurrentUser -Force
}

In [19]:
# return the functions available in Error-Utils and Text-Utils

Get-Command -Module Rad-Error-Utils
Get-Command -Module Rad-Text-Utils


CommandType     Name                                               Version    Source
-----------     ----                                               -------    ------
Function        Format-RadErrorMessage                             0.0        Rad-Error-Utils
Function        Get-RadErrorMessage                                0.0        Rad-Error-Utils
Function        Get-RadErrorMessages                               0.0        Rad-Error-Utils
Function        Import-RadErrorsFromYaml                           0.0        Rad-Error-Utils
Function        Invoke-TerminatingCommand                          0.0        Rad-Error-Utils
Function        Set-RadErrorMessages                               0.0        Rad-Error-Utils
Function        Confirm-LoggerIsEnabled                            0.0        Rad-Text-Utils
Function        Show-DebugOutput                                   0.0        Rad-Text-Utils
Function        Show-Error                                         0.0        R

### Get-RadErrorMessages

`Get-RadErrorMessages` returns all the error codes and messages that are available:

In [28]:
Get-RadErrorMessages


Name                           Value
----                           -----
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



### Get-RadErrorMessage

`Get-RadErrorMessage` returns a specific error message based on the error code passed in. If the error code is not found, it will return an error message stating that the error code was not found.

Some messages allow parameters to be passed in to customize the message. More information on this can be found in the [Format-RadErrorMessage](#Format-RadErrorMessage) section.

In [7]:
Get-RadErrorMessage -ErrorCode 101 -Parameters "param1","param2","param3"

Error 102: Invalid parameter: param1. Expected: param2. Actual: param3.. Error Code: 101


- If a message requires parameters but they are not provided, the function will return an error:

In [8]:
Get-RadErrorMessage -ErrorCode 101 

Exception: Error: The template requires 3 parameters, but only 0 were provided.


Error: Command failed: SubmitCode: Get-RadErrorMessage -ErrorCode 101 

- If the error code is not found, it will return a generic error message:

In [9]:
Get-RadErrorMessage -ErrorCode 200

An unexpected error occurred while executing the command. Error Code: 200


Error: Command failed: SubmitCode: Get-RadErrorMessage -ErrorCode 200

### Format-RadErrorMessage

`Format-RadErrorMessage` is used to format error messages in a consistent way to improve readability and maintainability.

It is also used to insert parameters into a message. It takes in a message and a list of parameters and returns the message with the parameters inserted. The parameters are inserted into the message in the order they are passed in in the list. Parameters are denoted in the message by `{}` and are required if they are present. Not all messages need to have parameters.

In [27]:
$ErrorMessages = Get-RadErrorMessages
$ErrorCode = "101"

Format-RadErrorMessage -Template $ErrorMessages[$ErrorCode] -Parameters "param1","param2","param3"

Error 101: Invalid parameter: [param1]. Expected: [param2]. Actual: [param3].


### Import-RadErrorsFromYaml

`Import-RadErrorsFromYaml` is used to import error codes and messages from a YAML file. It takes in a path to a YAML file and returns a hashtable of the error codes and messages. This allows you to add your own custom error codes and messages. The YAML file must be in the following format:

```yaml
  # common_errors.yaml
  errors:
    1000: "Unexpected error"
    1001: "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    1004: "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    1005: "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    1010: "Unsupported operation - The operation {0} is not supported."
```



In [ ]:
Import-RadErrorsFromYaml -YamlFile ".\samples\errors-example.yaml"

Then after importing the YAML file, you can use the `Get-RadErrorMessages` function to get the error messages:

In [50]:
Get-RadErrorMessages


Name                           Value
----                           -----
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



### Set-RadErrorMessages

`Set-RadErrorMessages` allows you to add new error messages. It takes in a hashtable of error codes and messages and adds them to the existing error messages. If the error code already exists, it will overwrite the existing error message.

For comparison purposes, these are the default error error messages available at the start:

In [76]:
# For comparison purposes, these are the default error error messages available

Get-RadErrorMessages


Name                           Value
----                           -----
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



Now, let's add a few new error messages:

In [77]:
$ErrorMessages = @{
    "1000" = "Unexpected error"
    "1001" = "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    "1004" = "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    "1005" = "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    "1010" = "Unsupported operation - The operation {0} is not supported."
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages


Name                           Value
----                           -----
1000                           Unexpected error
1001                           FileNotFoundError - Failure to locate or access a specified file or…
1004                           PermissionDeniedError - Insufficient permissions to perform the {0}…
1005                           TypeError - Mismatched data types or unsupported operations on a da…
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
1010                           Unsupported operation - The operation {0} is not supported.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



Finally, let's update an existing error message:

In [78]:
$ErrorMessages = @{
    "1000" = "A RAD error message"
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages


Name                           Value
----                           -----
1000                           A RAD error message
1001                           FileNotFoundError - Failure to locate or access a specified file or…
1004                           PermissionDeniedError - Insufficient permissions to perform the {0}…
1005                           TypeError - Mismatched data types or unsupported operations on a da…
101                            Error 102: Invalid parameter: {0}. Expected: {1}. Actual: {2}.
1010                           Unsupported operation - The operation {0} is not supported.
500                            An unexpected error occurred while executing the command: [{0}]. Er…



> There is an optional parameter `-Overwrite` that will overwrite all the existing error messages with the new error messages.

In [79]:
$ErrorMessages = @{
    "2000" = "Unexpected error"
    "2001" = "FileNotFoundError - Failure to locate or access a specified file or directory. [{0}]"
    "2004" = "PermissionDeniedError - Insufficient permissions to perform the {0} operation."
    "2005" = "TypeError - Mismatched data types or unsupported operations on a data type {0}."
    "2010" = "Unsupported operation - The operation {0} is not supported."
}

Set-RadErrorMessages -NewErrorMessages $ErrorMessages -Overwrite


Name                           Value
----                           -----
2000                           Unexpected error
2001                           FileNotFoundError - Failure to locate or access a specified file or…
2004                           PermissionDeniedError - Insufficient permissions to perform the {0}…
2005                           TypeError - Mismatched data types or unsupported operations on a da…
2010                           Unsupported operation - The operation {0} is not supported.



Now when you run Get-RadErrorMessages you will see the only new error messages:

In [80]:
Get-RadErrorMessages


Name                           Value
----                           -----
2000                           Unexpected error
2005                           TypeError - Mismatched data types or unsupported operations on a da…
2010                           Unsupported operation - The operation {0} is not supported.
2001                           FileNotFoundError - Failure to locate or access a specified file or…
2004                           PermissionDeniedError - Insufficient permissions to perform the {0}…

